#Misc.

In [ ]:
import pandas as pd
from sklearn.metrics import roc_curve,roc_auc_score,auc
from __future__ import absolute_import, division, print_function, unicode_literals
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from progressbar import ProgressBar
import math
import matplotlib.pyplot as plt
import statistics
import random
import sklearn
from numpy.random import seed
from tensorflow import random as tfRandom
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor
import warnings
import statistics
import gc
from sys import getsizeof
warnings.filterwarnings('ignore')



In [ ]:
debugMode = False
debugSeed = 1234
adultOnly = False
childrenOnly = False
activationOption = 'tanh'
doingTests = False
primaryOnly = False

#General

##Reading input from drive

In [ ]:
matrix = pd.read_table("../../data/Processed/bihourlyNegativeOne.csv",delimiter = ",")

demo = pd.read_table("../../data/Processed/demoInput.csv",delimiter = ",")
los = pd.read_table("../../data/Processed/los.csv",delimiter = ",")
mortality = pd.read_table("../../data/Processed/mortality.csv",delimiter = ",")
firstDayPrescriptionReduced = pd.read_table("../../data/Processed/firstDayPrescriptionReduced.csv",delimiter = ",")
secondDayPrescriptionReduced = pd.read_table("../../data/Processed/secondDayPrescriptionReduced.csv",delimiter = ",")
notesEmbedding = pd.read_table("../../data/Processed/notesInput.csv",delimiter=',',header=None)
aps3 = pd.read_table("../../data/Processed/aps3.csv",delimiter=",")
oasis = pd.read_table("../../data/Processed/oasis.csv",delimiter=",")
sofa = pd.read_table("../../data/Processed/sofa.csv",delimiter=",")
comorbidity = pd.read_table("../../data/Processed/ElixhauserCompressed.csv",delimiter=",")
if primaryOnly:
  diag = pd.read_table("../../data/Processed/primaryDiagnosis.csv",delimiter=",")
else:
  diag = pd.read_table("../../data/Processed/allDiagnosis.csv",delimiter=",")
del demo['Unnamed: 0']
del los['Unnamed: 0']
del mortality['Unnamed: 0']


In [ ]:
embeddingColNames = ["ICUID","SeqNum","TimeSinceLastNote","Case Management","Consult","General","Nursing","Nursing/other","Nutrition","Pharmacy","Physician ","Radiology","Rehab Services","Respiratory ","Social Work"]
embeddingColNames.extend(range(1,769))

In [ ]:
notesEmbedding.columns = embeddingColNames

In [ ]:
import csv

with open('../../data//ICUToHAMDDict.csv') as f:
    next(f)  # Skip the header
    reader = csv.reader(f, skipinitialspace=True)
    result = dict(reader)
    #print(result)
HADMDict = result

In [ ]:
pbar = ProgressBar()
matrixList = []
startCount = 0
currentID = matrix.iloc[0]["ICUID"]
endCount = 1
for i in pbar(range(1,matrix.shape[0])):
  if(currentID!=matrix.iloc[i]["ICUID"]):
    #print(endCount)
    matrixList.append(matrix.iloc[startCount:endCount])
    startCount = i
    endCount = i+1
    currentID = matrix.iloc[i]["ICUID"]
  else:
    endCount = endCount + 1;

In [ ]:
pbar = ProgressBar()
inputList = []
losIntList = []
ICUIDWONotesList = []
for mEle in pbar(matrixList):
  tempList = []
  ICUID = mEle.iloc[0]["ICUID"]
  #print(ICUID)
  tempList.append(mEle.drop(columns = ['ICUID','ChartDate','ChartTime'],errors='ignore'))
  tempList.append(demo.loc[demo['ICU.ID'] == ICUID].drop(columns = ['ICU.ID']))#,'Unnamed: 0']))
  tempLos = los.loc[los['ICU.ID'] == ICUID]['los']
  tempMortality = mortality.loc[mortality['ICU.ID']==ICUID]['mortality']
  tempPreFirstDay = firstDayPrescriptionReduced.loc[(firstDayPrescriptionReduced['ICUID']==ICUID)].drop(columns=['ICUID'])
  tempPreSecondDay = secondDayPrescriptionReduced.loc[(secondDayPrescriptionReduced['ICUID']==ICUID)].drop(columns=['ICUID'])
  if(tempLos.shape[0]!=1):
    continue
  tempLosList = []
  #######
  for i in range(3,15): 
    if math.ceil(tempLos) == i:
      tempLosList.append(True)
    else:
      tempLosList.append(False)
  if math.ceil(tempLos)>14:
    tempLosList.append(True)
  else:
    tempLosList.append(False)
  tempList.append(tempLosList)
  tempList.append(tempLos-2)
 ##########
  if(math.ceil(tempLos)<4):
    tempList.append(True)
  else:
    tempList.append(False)
  if(math.ceil(tempLos)<5):
    tempList.append(True)
  else:
    tempList.append(False)
  if(math.ceil(tempLos)<6):
    tempList.append(True)
  else:
    tempList.append(False)
  if(math.ceil(tempLos)<7):
    tempList.append(True)
  else:
    tempList.append(False)
  if(math.ceil(tempLos)<8):
    tempList.append(True)
  else:
    tempList.append(False)
  if(math.ceil(tempLos)<9):
    tempList.append(True)
  else:
    tempList.append(False)
  if(math.ceil(tempLos)<10):
    tempList.append(True)
  else:
    tempList.append(False)
  tempList.append(tempMortality)
  #prescription info
  tempList.append(tempPreFirstDay)
  tempList.append(tempPreSecondDay)
  losIntList.append(math.ceil(tempLos))
  #notes embedding
  validEmbeddings = notesEmbedding.loc[notesEmbedding["ICUID"]==ICUID]
  validEmbeddings = validEmbeddings.drop(columns=["ICUID","SeqNum"])
  tempList.append(validEmbeddings.values.tolist())
  #Drops cases without a single note
  if validEmbeddings.shape[0]==0:
    ICUIDWONotesList.append(ICUID)
    continue
  tempList.append(aps3.loc[aps3["icustay_id"]==ICUID]["apsiii"])
  HADMId = HADMDict[str(int(ICUID))]
  tempSave = comorbidity.loc[comorbidity['hadm_id'] == int(HADMId),:]
  tempSave = tempSave.drop(columns = ['hadm_id','subject_id'])
  tempList.append(tempSave)
  if (tempSave.shape[0]!=1):
    print(ICUID)
  tempSave = diag.loc[diag['HADM_ID'] == int(HADMId),:]
  tempSave = tempSave.drop(columns = ['HADM_ID'])
  tempList.append(tempSave)
  if (tempSave.shape[0]!=1):
    print(ICUID)
  #Testing stuff
  if adultOnly:
    if demo.loc[demo['ICU.ID'] == ICUID]['AgeAdm'].values[0]<18:
      continue
  if childrenOnly:
     if demo.loc[demo['ICU.ID'] == ICUID]['AgeAdm'].values[0]>18:
      continue   
  inputList.append(tempList)

In [ ]:
del matrixList
del demo
del matrix
del los
del mortality
del firstDayPrescriptionReduced
del secondDayPrescriptionReduced
del notesEmbedding
del aps3
del oasis
del sofa 
del diag
#del comorbidity

##Shuffle & sort

In [ ]:
def reshuffle():

  global inputList

  global inputDemoList
  global inputMatrixList
  global inputMatrixFirstDayList
  global inputMatrixSecondDayList
  global inputPrescriptionFirstDayList
  global inputPrescriptionSecondDayList
  global inputNotesEmbeddingList
  global inputSAPSList
  global inputOASISList
  global inputSOFAList
  global inputComorbidityList
  global inputDiagList
  global outputLoSList
  global outputNumList
  global outputBool3List
  global outputBool4List
  global outputBool5List
  global outputBool6List
  global outputBool7List
  global outputBool8List
  global outputBool9List
  global outputBoolMortalList
  global TinputDemoList
  global TinputMatrixList
  global TinputMatrixFirstDayList
  global TinputMatrixSecondDayList
  global TinputPrescriptionFirstDayList
  global TinputPrescriptionSecondDayList
  global TinputNotesEmbeddingList
  global TinputSAPSList
  global TinputOASISList
  global TinputSOFAList
  global TinputComorbidityList
  global TinputDiagList
  global ToutputLoSList
  global ToutputNumList
  global ToutputBool3List
  global ToutputBool4List
  global ToutputBool5List
  global ToutputBool6List
  global ToutputBool7List
  global ToutputBool8List
  global ToutputBool9List
  global ToutputBoolMortalList

  global inputDemoArray
  global inputMatrixArray
  global inputMatrixFirstDayArray
  global inputMatrixSecondDayArray
  global inputPrescriptionFirstDayArray
  global inputPrescriptionSecondDayArray
  global inputNotesEmbeddingArray
  global inputSAPSArray
  global inputOASISArray
  global inputSOFAArray
  global inputComorbidityArray
  global inputDiagArray
  global outputLoSArray
  global outputNumArray
  global outputBool3Array
  global outputBool4Array
  global outputBool5Array
  global outputBool6Array
  global outputBool7Array
  global outputBool8Array
  global outputBool9Array
  global outputBoolMortalArray
  global TinputDemoArray
  global TinputMatrixArray
  global TinputMatrixFirstDayArray
  global TinputMatrixSecondDayArray
  global TinputPrescriptionFirstDayArray
  global TinputPrescriptionSecondDayArray
  global TinputNotesEmbeddingArray
  global TinputSAPSArray
  global TinputOASISArray
  global TinputSOFAArray
  global TinputComorbidityArray
  global TinputDiagArray
  global ToutputLoSArray
  global ToutputNumArray
  global ToutputBool3Array
  global ToutputBool4Array
  global ToutputBool5Array
  global ToutputBool6Array
  global ToutputBool7Array
  global ToutputBool8Array
  global ToutputBool9Array
  global ToutputBoolMortalArray

  inputDemoList = []
  inputMatrixList = []
  inputMatrixFirstDayList = []
  inputMatrixSecondDayList = []
  inputPrescriptionFirstDayList = []
  inputPrescriptionSecondDayList = []
  inputNotesEmbeddingList = []
  inputSAPSList = []
  inputOASISList = []
  inputSOFAList = []
  inputComorbidityList = []
  inputDiagList = []
  outputLoSList = []
  outputNumList = []
  outputBool3List = []
  outputBool4List = []
  outputBool5List = []
  outputBool6List = []
  outputBool7List = []
  outputBool8List = []
  outputBool9List = []
  outputBoolMortalList = []

  TinputDemoList = []
  TinputMatrixList = []
  TinputMatrixFirstDayList = []
  TinputMatrixSecondDayList = []
  TinputPrescriptionFirstDayList = []
  TinputPrescriptionSecondDayList = []
  TinputNotesEmbeddingList = []
  TinputSAPSList = []
  TinputOASISList = []
  TinputSOFAList = []
  TinputComorbidityList = []
  TinputDiagList = []
  ToutputLoSList = []
  ToutputNumList = []
  ToutputBool3List = []
  ToutputBool4List = []
  ToutputBool5List = []
  ToutputBool6List = []
  ToutputBool7List = []
  ToutputBool8List = []
  ToutputBool9List = []
  ToutputBoolMortalList = []

  random.shuffle(inputList)
  count = [1,1,1,1,1,1,1,1,1,1,1,1,1]
  testList = []
  trainList = []
  for i in inputList:
    if not float(i[3]) > 0:
      continue
    LoSLabel = math.ceil(i[3])
    if LoSLabel>13:
      LoSLabel = 13
    if count[LoSLabel-1] < 3:
      testList.append(i)
    else:
      trainList.append(i)
    count[LoSLabel-1] = count[LoSLabel-1] + 1
    if count[LoSLabel-1] == 11:
      count[LoSLabel-1] = 1

  for i in trainList:
    #print(i[0])
    inputDemoList.append(i[1].to_numpy())
    inputMatrixList.append(i[0].to_numpy())
    inputMatrixFirstDayList.append(i[0].to_numpy()[0:12])
    inputMatrixSecondDayList.append(i[0].to_numpy()[12:])
    outputLoSList.append(np.asarray(i[2]))
    outputNumList.append(float(i[3]))
    outputBool3List.append(i[4])
    outputBool4List.append(i[5])
    outputBool5List.append(i[6])
    outputBool6List.append(i[7])
    outputBool7List.append(i[8])
    outputBool8List.append(i[9])
    outputBool9List.append(i[10])
    outputBoolMortalList.append(i[11])
    inputPrescriptionFirstDayList.append(i[12])
    inputPrescriptionSecondDayList.append(i[13])
    inputNotesEmbeddingList.append(i[14])
    inputSAPSList.append(i[15])
    inputOASISList.append(i[16])
    inputSOFAList.append(i[17])
    inputComorbidityList.append(i[18])
    inputDiagList.append(i[19])
  for i in testList:
    TinputDemoList.append(i[1].to_numpy())
    TinputMatrixList.append(i[0].to_numpy())
    TinputMatrixFirstDayList.append(i[0].to_numpy()[0:12])
    TinputMatrixSecondDayList.append(i[0].to_numpy()[12:])
    ToutputLoSList.append(np.asarray(i[2]))
    ToutputNumList.append(float(i[3]))
    ToutputBool3List.append(i[4])
    ToutputBool4List.append(i[5])
    ToutputBool5List.append(i[6])
    ToutputBool6List.append(i[7])
    ToutputBool7List.append(i[8])
    ToutputBool8List.append(i[9])
    ToutputBool9List.append(i[10])
    ToutputBoolMortalList.append(i[11])
    TinputPrescriptionFirstDayList.append(i[12])
    TinputPrescriptionSecondDayList.append(i[13])
    TinputNotesEmbeddingList.append(i[14])
    TinputSAPSList.append(i[15])
    TinputOASISList.append(i[16])
    TinputSOFAList.append(i[17])
    TinputComorbidityList.append(i[18])
    TinputDiagList.append(i[19])

    
  inputDemoArray = np.stack(inputDemoList)
  inputMatrixArray = np.stack(inputMatrixList)
  inputMatrixFirstDayArray = np.stack(inputMatrixFirstDayList)
  inputMatrixSecondDayArray = np.stack(inputMatrixSecondDayList)
  inputPrescriptionFirstDayArray = np.stack(inputPrescriptionFirstDayList)
  inputPrescriptionSecondDayArray = np.stack(inputPrescriptionSecondDayList)
  inputNotesEmbeddingArray = keras.preprocessing.sequence.pad_sequences(inputNotesEmbeddingList)#tf.ragged.constant(inputNotesEmbeddingList,ragged_rank=1)
  inputSAPSArray = np.stack(inputSAPSList)
  inputOASISArray = np.stack(inputOASISList)
  inputSOFAArray = np.stack(inputSOFAList)
  inputComorbidityArray = np.stack(inputComorbidityList)
  inputDiagArray = np.stack(inputDiagList)
  outputLoSArray = np.stack(outputLoSList)
  outputNumArray = np.stack(outputNumList)
  outputBool3Array = np.stack(outputBool3List)
  outputBool4Array = np.stack(outputBool4List)
  outputBool5Array = np.stack(outputBool5List)
  outputBool6Array = np.stack(outputBool6List)
  outputBool7Array = np.stack(outputBool7List)
  outputBool8Array = np.stack(outputBool8List)
  outputBool9Array = np.stack(outputBool9List)
  outputBoolMortalArray = np.stack(outputBoolMortalList)

  TinputDemoArray = np.stack(TinputDemoList)
  TinputMatrixArray = np.stack(TinputMatrixList)
  TinputMatrixFirstDayArray = np.stack(TinputMatrixFirstDayList)
  TinputMatrixSecondDayArray = np.stack(TinputMatrixSecondDayList)
  TinputPrescriptionFirstDayArray = np.stack(TinputPrescriptionFirstDayList)
  TinputPrescriptionSecondDayArray = np.stack(TinputPrescriptionSecondDayList)
  TinputNotesEmbeddingArray = keras.preprocessing.sequence.pad_sequences(TinputNotesEmbeddingList)#tf.ragged.constant(TinputNotesEmbeddingList,ragged_rank=1)
  TinputSAPSArray = np.stack(TinputSAPSList)
  TinputOASISArray = np.stack(TinputOASISList)
  TinputSOFAArray = np.stack(TinputSOFAList)
  TinputComorbidityArray = np.stack(TinputComorbidityList)
  TinputDiagArray = np.stack(TinputDiagList)
  ToutputLoSArray = np.stack(ToutputLoSList)
  ToutputNumArray = np.stack(ToutputNumList)
  ToutputBool3Array = np.stack(ToutputBool3List)
  ToutputBool4Array = np.stack(ToutputBool4List)
  ToutputBool5Array = np.stack(ToutputBool5List)
  ToutputBool6Array = np.stack(ToutputBool6List)
  ToutputBool7Array = np.stack(ToutputBool7List)
  ToutputBool8Array = np.stack(ToutputBool8List)
  ToutputBool9Array = np.stack(ToutputBool9List)
  ToutputBoolMortalArray = np.stack(ToutputBoolMortalList)

  outputLoSArray = outputLoSArray.reshape(outputLoSArray.shape[0],1,outputLoSArray.shape[1])
  ToutputLoSArray = ToutputLoSArray.reshape(ToutputLoSArray.shape[0],1,ToutputLoSArray.shape[1])
  outputLoSArray = outputLoSArray.reshape(outputLoSArray.shape[0],outputLoSArray.shape[2])
  ToutputLoSArray = ToutputLoSArray.reshape(ToutputLoSArray.shape[0],ToutputLoSArray.shape[2])

  del inputDemoList 
  del inputMatrixList 
  del inputMatrixFirstDayList 
  del inputMatrixSecondDayList 
  del inputPrescriptionFirstDayList 
  del inputPrescriptionSecondDayList 
  del inputNotesEmbeddingList
  del inputSAPSList
  del inputOASISList
  del inputSOFAList
  del outputLoSList 
  del outputNumList 
  del outputBool3List 
  del outputBool4List 
  del outputBool5List 
  del outputBool6List 
  del outputBool7List 
  del outputBool8List 
  del outputBool9List 
  del outputBoolMortalList 

  del TinputDemoList 
  del TinputMatrixList 
  del TinputMatrixFirstDayList 
  del TinputMatrixSecondDayList 
  del TinputPrescriptionFirstDayList 
  del TinputPrescriptionSecondDayList 
  del TinputNotesEmbeddingList 
  del TinputSAPSList
  del TinputOASISList
  del TinputSOFAList
  del ToutputLoSList 
  del ToutputNumList 
  del ToutputBool3List 
  del ToutputBool4List 
  del ToutputBool5List 
  del ToutputBool6List 
  del ToutputBool7List 
  del ToutputBool8List 
  del ToutputBool9List 
  del ToutputBoolMortalList 
  gc.collect()

In [ ]:
seed(2110)
tfRandom.set_seed(2110)
random.seed(2110)

reshuffle()

In [ ]:
def runModels(includeAdmin = False, 
              includeVitals = False, 
              includePrescriptionPCAs = False,
              bucketOutput = False,
              mortalityOutput = False,
              numericOutput = False,
              rnnMode = 'GRU'):
  reshuffle()
  rnnModel, rnnResult = RNNRuns(includeAdmin,includeVitals,includePrescriptionPCAs,bucketOutput,mortalityOutput,numericOutput,rnnMode)
  cnnModel, cnnResult = CNNRuns(includeAdmin,includeVitals,includePrescriptionPCAs,bucketOutput,mortalityOutput,numericOutput)
  randomForestResultDict = randomForestRuns(includeAdmin,includeVitals,includePrescriptionPCAs)
  returnDict = {}
  returnDict['RNN'] = rnnResult
  returnDict['CNN'] = cnnResult
  returnDict['RF'] = randomForestResultDict
  return(returnDict)



In [ ]:
def RunModelsAllOutput(includeAdmin = True, includeVitals = True, includePrescriptionPCAs = True, runCount = 10):
  pbar = ProgressBar()
  returnList = []
  for i in pbar(range(0,runCount)):
    returnVal = runModels(includeAdmin, 
                  includeVitals, 
                  includePrescriptionPCAs,
                  bucketOutput = True,
                  mortalityOutput = True,
                  numericOutput = True,
                  rnnMode = 'GRU')
    returnList.append(returnVal)

  RNNBucketAUCList = []
  RNNBucketKapList = []
  RNNBucketACCList = []
  RNNMortalACCList = []
  RNNMortalAUCList = []
  RNNNumeriMSEList = []
  RNNNumeriMAEList = []

  CNNBucketAUCList = []
  CNNBucketKapList = []
  CNNBucketACCList = []
  CNNMortalACCList = []
  CNNMortalAUCList = []
  CNNNumeriMSEList = []
  CNNNumeriMAEList = []

  RF_BucketAUCList = []
  RF_BucketKapList = []
  RF_BucketACCList = []
  RF_MortalACCList = []
  RF_MortalAUCList = []
  RF_NumeriMSEList = []
  RF_NumeriMAEList = []

  for i in returnList:
    RNNBucketAUCList.append(i['RNN']['Bucket_Output_auc'])
    RNNBucketKapList.append(i['RNN']['Bucket_Output_kappa'])
    RNNBucketACCList.append(i['RNN']['Bucket_Output_accuracy'])
    RNNMortalACCList.append(i['RNN']['Mortality_Output_accuracy'])
    RNNMortalAUCList.append(i['RNN']['Mortality_Output_auc'])
    RNNNumeriMSEList.append(i['RNN']['Numerical_Output_mean_squared_error'])
    RNNNumeriMAEList.append(i['RNN']['Numerical_Output_mean_absolute_error'])

    CNNBucketAUCList.append(i['CNN']['Bucket_Output_auc'])
    CNNBucketKapList.append(i['CNN']['Bucket_Output_kappa'])
    CNNBucketACCList.append(i['CNN']['Bucket_Output_accuracy'])
    CNNMortalACCList.append(i['CNN']['Mortality_Output_accuracy'])
    CNNMortalAUCList.append(i['CNN']['Mortality_Output_auc'])
    CNNNumeriMSEList.append(i['CNN']['Numerical_Output_mean_squared_error'])
    CNNNumeriMAEList.append(i['CNN']['Numerical_Output_mean_absolute_error'])

    RF_BucketAUCList.append(i["RF"]['aucLoS'])
    RF_BucketKapList.append(i["RF"]['kapLoS'])
    RF_BucketACCList.append(i["RF"]['accLoS'])
    RF_MortalACCList.append(i["RF"]['accMor'])
    RF_MortalAUCList.append(i["RF"]['aucMor'])
    RF_NumeriMSEList.append(i["RF"]['MSENum'])
    RF_NumeriMAEList.append(i["RF"]['MAENum'])
  print('Admin. Data: %r, Vital signs: %r, Prescriptions: %r' % (includeAdmin, includeVitals, includePrescriptionPCAs))
  print("-----Bucket output Accuracy:")
  print("RNN Mean: %.6f //// Stdev: %.6f" % (statistics.mean(RNNBucketACCList), statistics.stdev(RNNBucketACCList)))
  print("CNN Mean: %.6f //// Stdev: %.6f" % (statistics.mean(CNNBucketACCList), statistics.stdev(CNNBucketACCList)))
  print("RF  Mean: %.6f //// Stdev: %.6f" % (statistics.mean(RF_BucketACCList), statistics.stdev(RF_BucketACCList)))

  print("-----Bucket output AUC:")
  print("RNN Mean: %.6f //// Stdev: %.6f" % (statistics.mean(RNNBucketAUCList), statistics.stdev(RNNBucketAUCList)))
  print("CNN Mean: %.6f //// Stdev: %.6f" % (statistics.mean(CNNBucketAUCList), statistics.stdev(CNNBucketAUCList)))
  print("RF  Mean: %.6f //// Stdev: %.6f" % (statistics.mean(RF_BucketAUCList), statistics.stdev(RF_BucketAUCList)))

  print("-----Bucket output Kappa:")
  print("RNN Mean: %.6f //// Stdev: %.6f" % (statistics.mean(RNNBucketKapList), statistics.stdev(RNNBucketKapList)))
  print("CNN Mean: %.6f //// Stdev: %.6f" % (statistics.mean(CNNBucketKapList), statistics.stdev(CNNBucketKapList)))
  print("RF  Mean: %.6f //// Stdev: %.6f" % (statistics.mean(RF_BucketKapList), statistics.stdev(RF_BucketKapList)))

  print("-----Mortality output Accuracy:")
  print("RNN Mean: %.6f //// Stdev: %.6f" % (statistics.mean(RNNMortalACCList), statistics.stdev(RNNMortalACCList)))
  print("CNN Mean: %.6f //// Stdev: %.6f" % (statistics.mean(CNNMortalACCList), statistics.stdev(CNNMortalACCList)))
  print("RF  Mean: %.6f //// Stdev: %.6f" % (statistics.mean(RF_MortalACCList), statistics.stdev(RF_MortalACCList)))

  print("-----Mortality output AUC:")
  print("RNN Mean: %.6f //// Stdev: %.6f" % (statistics.mean(RNNMortalAUCList), statistics.stdev(RNNMortalAUCList)))
  print("CNN Mean: %.6f //// Stdev: %.6f" % (statistics.mean(CNNMortalAUCList), statistics.stdev(CNNMortalAUCList)))
  print("RF  Mean: %.6f //// Stdev: %.6f" % (statistics.mean(RF_MortalAUCList), statistics.stdev(RF_MortalAUCList)))

  print("-----Numeric output MAE:")
  print("RNN Mean: %.6f //// Stdev: %.6f" % (statistics.mean(RNNNumeriMAEList), statistics.stdev(RNNNumeriMAEList)))
  print("CNN Mean: %.6f //// Stdev: %.6f" % (statistics.mean(CNNNumeriMAEList), statistics.stdev(CNNNumeriMAEList)))
  print("RF  Mean: %.6f //// Stdev: %.6f" % (statistics.mean(RF_NumeriMAEList), statistics.stdev(RF_NumeriMAEList)))

  print("-----Numeric output MSE:")
  print("RNN Mean: %.6f //// Stdev: %.6f" % (statistics.mean(RNNNumeriMSEList), statistics.stdev(RNNNumeriMSEList)))
  print("CNN Mean: %.6f //// Stdev: %.6f" % (statistics.mean(CNNNumeriMSEList), statistics.stdev(CNNNumeriMSEList)))
  print("RF  Mean: %.6f //// Stdev: %.6f" % (statistics.mean(RF_NumeriMSEList), statistics.stdev(RF_NumeriMSEList)))

##Current Functions

In [ ]:
def RunModelsNumericOutput(includeAdmin = True, includeDiag = True, includeVitals = True, includePrescriptionPCAs = True, includeEmbeddings = False, runCount = 10):
  pbar = ProgressBar()
  returnList = []
  for i in pbar(range(0,runCount)):
    tempDict = {}
    rnnModel, rnnResult = RNNRuns(includeAdmin,includeDiag,includeVitals,includePrescriptionPCAs,includeEmbeddings,False,False,True,"LSTM")
    #cnnModel, cnnResult = CNNRuns(includeAdmin,includeVitals,includePrescriptionPCAs,includeEmbeddings,bucketOutput = False,mortalityOutput = False,numericOutput = True)
    tempDict['RNN'] = rnnResult
    #tempDict['CNN'] = cnnResult
    returnList.append(tempDict)


  RNNNumeriMSEList = []
  RNNNumeriMAEList = []

  #CNNNumeriMSEList = []
  #CNNNumeriMAEList = []


  for i in returnList:
    RNNNumeriMSEList.append(i['RNN']['mean_squared_error'])
    RNNNumeriMAEList.append(i['RNN']['mean_absolute_error']) 

    #CNNNumeriMSEList.append(i['CNN']['mean_squared_error'])
    #CNNNumeriMAEList.append(i['CNN']['mean_absolute_error']) 

  print("-----Numeric output MAE:")
  print("RNN Mean: %.6f //// Stdev: %.6f" % (statistics.mean(RNNNumeriMAEList), statistics.stdev(RNNNumeriMAEList)))
  #print("CNN Mean: %.6f //// Stdev: %.6f" % (statistics.mean(CNNNumeriMAEList), statistics.stdev(CNNNumeriMAEList)))

  print("-----Numeric output MSE:")
  print("RNN Mean: %.6f //// Stdev: %.6f" % (statistics.mean(RNNNumeriMSEList), statistics.stdev(RNNNumeriMSEList)))
  #print("CNN Mean: %.6f //// Stdev: %.6f" % (statistics.mean(CNNNumeriMSEList), statistics.stdev(CNNNumeriMSEList)))

In [ ]:
def rfOutput(includeAdmin = True, includeVitals = True, includePrescriptionPCAs = True, runCount = 10):
  pbar = ProgressBar()
  returnList = []
  for i in pbar(range(0,runCount)):
    tempDict = {}
    rfReturn = randomForestRuns(includeAdmin, includeVitals, includePrescriptionPCAs)
    returnList.append(rfReturn)

  RF_BucketAUCList = []
  RF_BucketKapList = []
  RF_BucketACCList = []
  RF_MortalACCList = []
  RF_MortalAUCList = []
  RF_NumeriMSEList = []
  RF_NumeriMAEList = []

  for i in returnList:
    RF_BucketAUCList.append(i['aucLoS'])
    RF_BucketKapList.append(i['kapLoS'])
    RF_BucketACCList.append(i['accLoS'])
    RF_MortalACCList.append(i['accMor'])
    RF_MortalAUCList.append(i['aucMor'])
    RF_NumeriMSEList.append(i['MSENum'])
    RF_NumeriMAEList.append(i['MAENum'])

  print('Admin. Data: %r, Vital signs: %r, Prescriptions: %r' % (includeAdmin, includeVitals, includePrescriptionPCAs))
  print("-----Bucket output Accuracy:")
  print("RF  Mean: %.6f //// Stdev: %.6f" % (statistics.mean(RF_BucketACCList), statistics.stdev(RF_BucketACCList)))

  print("-----Bucket output AUC:")
  print("RF  Mean: %.6f //// Stdev: %.6f" % (statistics.mean(RF_BucketAUCList), statistics.stdev(RF_BucketAUCList)))

  print("-----Bucket output Kappa:")
  print("RF  Mean: %.6f //// Stdev: %.6f" % (statistics.mean(RF_BucketKapList), statistics.stdev(RF_BucketKapList)))

  print("-----Mortality output Accuracy:")
  print("RF  Mean: %.6f //// Stdev: %.6f" % (statistics.mean(RF_MortalACCList), statistics.stdev(RF_MortalACCList)))

  print("-----Mortality output AUC:")
  print("RF  Mean: %.6f //// Stdev: %.6f" % (statistics.mean(RF_MortalAUCList), statistics.stdev(RF_MortalAUCList)))

  print("-----Numeric output MAE:")
  print("RF  Mean: %.6f //// Stdev: %.6f" % (statistics.mean(RF_NumeriMAEList), statistics.stdev(RF_NumeriMAEList)))

  print("-----Numeric output MSE:")
  print("RF  Mean: %.6f //// Stdev: %.6f" % (statistics.mean(RF_NumeriMSEList), statistics.stdev(RF_NumeriMSEList)))

In [ ]:
def binaryRun(includeAdmin = False, includeDiag = False, includeVitals = False,  includePrescriptionPCAs = False, includeEmbeddings = False, binaryDate = 3, runCount = 10):
  pbar = ProgressBar()
  returnList = []
  for i in pbar(range(0,runCount)):
    returnList.append(binarySingleRun(includeAdmin, includeDiag, includeVitals,  includePrescriptionPCAs, includeEmbeddings, binaryDate))
    tf.keras.backend.clear_session()
    #tf.reset_default_graph()
  dlBinaryACCList = []
  dlBinaryAUCList = []
  dlBinaryMSEList = []
  dlBinaryMAEList = []

  for i in returnList:
    dlBinaryACCList.append(i['accuracy'])
    dlBinaryAUCList.append(i['auc'])
    dlBinaryMSEList.append(i['mse'])
    dlBinaryMAEList.append(i['mae']) 


  print("\n-----Binary Accuracy for exiting on %d day: " % binaryDate)
  print("Mean: %.3f //// Stdev: %.3f" % (statistics.mean(dlBinaryACCList), statistics.stdev(dlBinaryACCList)))

  print("\n-----Binary AUC for exiting on %d day: " % binaryDate)
  print("Mean: %.3f //// Stdev: %.3f" % (statistics.mean(dlBinaryAUCList), statistics.stdev(dlBinaryAUCList)))

  print("\n-----Binary MSE for exiting on %d day: " % binaryDate)
  print("Mean: %.3f //// Stdev: %.3f" % (statistics.mean(dlBinaryMSEList), statistics.stdev(dlBinaryMSEList)))
  
  print("\n-----Binary MAE for exiting on %d day: " % binaryDate)
  print("Mean: %.3f //// Stdev: %.3f" % (statistics.mean(dlBinaryMAEList), statistics.stdev(dlBinaryMAEList)))



In [ ]:
def binarySingleRun(includeAdmin = False, includeDiag = False, includeVitals = False,  includePrescriptionPCAs = False, includeEmbeddings = False, binaryDate = 3):
  reshuffle()
  tf.keras.backend.clear_session()
  outputVarList = []
  ToutputVarList = []
  if(binaryDate > 15 or binaryDate < 3):
    print("error")
    return()

  for i in outputNumArray:
    if i > (binaryDate - 2):
      outputVarList.append(False)
    else:
      outputVarList.append(True)
  for i in ToutputNumArray:
    if i > (binaryDate - 2):
      ToutputVarList.append(False)
    else:
      ToutputVarList.append(True)
  
  outputVarList = np.array(outputVarList)
  ToutputVarList = np.array(ToutputVarList)

  #dlModel = RNNBuildBinaryModel(includeAdmin,includeDiag, includeVitals, includePrescriptionPCAs,includeEmbeddings,rnnMode = 'LSTM')
  dlModel, dlResult = RNNBinaryRuns(outputVarList, ToutputVarList, includeAdmin, includeDiag, includeVitals, includePrescriptionPCAs,includeEmbeddings,rnnMode = 'LSTM')
  del dlModel
  gc.collect()
  print(dlResult)
  return(dlResult)

In [ ]:
def seperatIO(inputVarList, outputVar, categoryList):
  seperateList = []
  for i in categoryList:
    tempList = []
    for j in inputVarList:
      tempList.append(np.take(j,i,0))
    tempList.append(outputVar[i])
    seperateList.append(tempList)
  return seperateList

#RNN Functions

##Others

In [ ]:
def RNNBuildModel(includeAdmin = False, 
                  includeDiag = False,
                 includeVitals = False, 
                 includePrescriptionPCAs = False, 
                 includeEmbeddings = False,
                 bucketOutput = False, 
                 mortalityOutput = False, 
                 numericOutput = False, 
                 rnnMode = 'GRU'):

  if not (includeAdmin or includeDiag or includeVitals or includePrescriptionPCAs or includeEmbeddings):
    return("Error: no input")

  tf.keras.backend.clear_session()
  if debugMode:
    seed(debugSeed)
    tfRandom.set_seed(debugSeed) 

  featureCount = 512

  if includeVitals:
    matrixFirstDayInput = keras.Input(shape= (12,8),dtype = 'float32',name = 'Vital_Signs_First_day_Input')
    matrixSecondDayInput = keras.Input(shape= (12,8),dtype = 'float32',name = 'Vital_Signs_Second_day_Input')
    #first day
    if rnnMode == 'GRU':
      firstDayRNNLayer = keras.layers.GRU(featureCount,input_shape = (12,8), name = "first_day_RNN_layer")(matrixFirstDayInput)
    else:
      firstDayRNNLayer = keras.layers.LSTM(featureCount,input_shape = (12,8), name = "first_day_RNN_layer")(matrixFirstDayInput)
    firstDaySDenseLayer = keras.layers.Dense(featureCount, name = "first_day_Dense_layer")(firstDayRNNLayer)
    firstDayReshapeLayer = keras.layers.Reshape((1,featureCount), name = "first_day_Reshape_layer")(firstDaySDenseLayer)
    #second day
    if rnnMode == 'GRU':
      secondDayRNNLayer = keras.layers.GRU(featureCount,input_shape = (12,8), name = "second_day_RNN_layer")(matrixSecondDayInput)
    else:
      secondDayRNNLayer = keras.layers.LSTM(featureCount,input_shape = (12,8), name = "second_day_RNN_layer")(matrixSecondDayInput)
    secondDaySDenseLayer = keras.layers.Dense(featureCount, name = "second_day_Dense_layer")(secondDayRNNLayer)
    secondDayReshapeLayer = keras.layers.Reshape((1,featureCount), name = "second_day_Reshape_layer")(secondDaySDenseLayer)
    #combine days
  if includePrescriptionPCAs:
    prescriptionFirstDayInput = keras.Input(shape = (1,7),dtype = "float32",name = 'Prescription_First_Day_Input')
    prescriptionSecondDayInput = keras.Input(shape = (1,7),dtype = "float32",name = 'Prescription_Second_Day_Input')
    if includeVitals:
      firstDayMergeLayer = keras.layers.concatenate([firstDayReshapeLayer,prescriptionFirstDayInput],name = 'first_Day_Prescription_Merge_Layer') #1 row
      secondDayMergeLayer = keras.layers.concatenate([secondDayReshapeLayer,prescriptionSecondDayInput],name = 'second_Day_Prescription_Merge_Layer') #1 row
      firstSecondDaymergeLayer = keras.layers.concatenate([firstDayMergeLayer,secondDayMergeLayer],name = 'first_Second_day_Merge_Layer', axis = 1) #2 rows
    else:
      firstSecondDaymergeLayer = keras.layers.concatenate([prescriptionFirstDayInput,prescriptionSecondDayInput],name = 'first_Second_day_Merge_Layer', axis = 1) #2 rows
  else:
    if includeVitals:
      firstSecondDaymergeLayer = keras.layers.concatenate([firstDayReshapeLayer,secondDayReshapeLayer],name = 'first_Second_day_Merge_Layer', axis = 1) #2 rows
  if includeEmbeddings:
    embeddingInput = keras.Input(shape = (None,781),dtype = 'float32',name = 'Embedding_Input')
    maskingLayer = keras.layers.Masking(mask_value = 0)(embeddingInput)
    if rnnMode == 'GRU':
      embeddingRNNLayer = keras.layers.GRU(featureCount, input_shape = (None,781),name = "Embedding_RNN_Layer")(maskingLayer)
    else:
      embeddingRNNLayer = keras.layers.LSTM(featureCount, input_shape = (None,781),name = "Embedding_RNN_Layer")(maskingLayer)
    embeddingReshapeLayer = keras.layers.Reshape((1,featureCount), name = "embedding_Reshape_layer")(embeddingRNNLayer)
  if (includeVitals or includePrescriptionPCAs):
    if rnnMode == 'GRU':
      combinedRNNLayer = keras.layers.GRU(featureCount, name = "combined_RNN_layer")(firstSecondDaymergeLayer)
    else:
      combinedRNNLayer = keras.layers.LSTM(featureCount, name = "combined_RNN_layer")(firstSecondDaymergeLayer)
    combinedDenseLayer = keras.layers.Dense(featureCount,name = "combined_Dense_layer")(combinedRNNLayer)
    combinedReshapeLayer = keras.layers.Reshape((1,featureCount), name = "combined_Reshape_layer")(combinedDenseLayer)
    #hiddenLayer1 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_1')(combinedReshapeLayer)
    if includeAdmin:
      demoInput = keras.Input(shape = (1,73),dtype = "float32",name = 'Admin_Data_Input')
      adminDataRNNMergeLayer = keras.layers.concatenate([combinedReshapeLayer,demoInput],name = 'admin_Data_RNN_Merge_Layer')
      if includeEmbeddings:
        if includeDiag:
          diagInput = keras.Input(shape = (1,18), dtype = 'float32', name = 'Diag_Data_Input')
          diagMergeLayer = keras.layers.concatenate([adminDataRNNMergeLayer,diagInput], name = 'diag_Merge_Layer')
          embeddingMergeLayer = keras.layers.concatenate([diagMergeLayer,embeddingReshapeLayer], name = 'embedding_Merge_Layer')
        else:
          embeddingMergeLayer = keras.layers.concatenate([adminDataRNNMergeLayer,embeddingReshapeLayer], name = 'embedding_Merge_Layer')
        hiddenLayer1 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_1')(embeddingMergeLayer)
      else:
        if includeDiag:
          diagInput = keras.Input(shape = (1,18), dtype = 'float32', name = 'Diag_Data_Input')
          diagMergeLayer = keras.layers.concatenate([adminDataRNNMergeLayer,diagInput], name = 'diag_Merge_Layer')
          hiddenLayer1 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_1')(diagMergeLayer)
        else:
          hiddenLayer1 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_1')(adminDataRNNMergeLayer)
    else:
      if includeEmbeddings:
        if includeDiag:
          diagInput = keras.Input(shape = (1,18), dtype = 'float32', name = 'Diag_Data_Input')
          diagMergeLayer = keras.layers.concatenate([combinedReshapeLayer,diagInput], name = 'diag_Merge_Layer')
          embeddingMergeLayer = keras.layers.concatenate([diagMergeLayer,embeddingReshapeLayer], name = 'embedding_Merge_Layer')
        else:
          embeddingMergeLayer = keras.layers.concatenate([combinedReshapeLayer,embeddingReshapeLayer], name = 'embedding_Merge_Layer')
        hiddenLayer1 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_1')(embeddingMergeLayer)
      else:
        if includeDiag:
          diagInput = keras.Input(shape = (1,18), dtype = 'float32', name = 'Diag_Data_Input')
          diagMergeLayer = keras.layers.concatenate([combinedReshapeLayer,diagInput], name = 'diag_Merge_Layer')
          hiddenLayer1 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_1')(diagMergeLayer)
        else:
          hiddenLayer1 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_1')(combinedReshapeLayer)
  else:
      demoInput = keras.Input(shape = (1,73),dtype = "float32",name = 'Admin_Data_Input')
      if includeEmbeddings:
        if includeDiag:
          diagInput = keras.Input(shape = (1,18), dtype = 'float32', name = 'Diag_Data_Input')
          diagMergeLayer = keras.layers.concatenate([demoInput,diagInput], name = 'diag_Merge_Layer')
          embeddingMergeLayer = keras.layers.concatenate([diagMergeLayer ,embeddingReshapeLayer], name = 'embedding_Merge_Layer')
        else:
          embeddingMergeLayer = keras.layers.concatenate([demoInput,embeddingReshapeLayer], name = 'embedding_Merge_Layer')
        hiddenLayer1 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_1')(embeddingMergeLayer)
      else:
        if includeDiag:
          diagInput = keras.Input(shape = (1,18), dtype = 'float32', name = 'Diag_Data_Input')
          diagMergeLayer = keras.layers.concatenate([demoInput,diagInput], name = 'diag_Merge_Layer')
          hiddenLayer1 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_1')(diagMergeLayer)
        else:
          hiddenLayer1 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_1')(demoInput)
  if includeEmbeddings and not includeAdmin and not includePrescriptionPCAs and not includeVitals:
    if includeDiag:
      diagInput = keras.Input(shape = (1,18), dtype = 'float32', name = 'Diag_Data_Input')
      diagMergeLayer = keras.layers.concatenate([embeddingReshapeLayer,diagInput], name = 'diag_Merge_Layer')
      hiddenLayer1 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_1')(diagMergeLayer)
    else:
      hiddenLayer1 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_1')(embeddingReshapeLayer)
  if includeDiag and not includeEmbeddings and not includeAdmin and not includePrescriptionPCAs and not includeVitals:
    diagInput = keras.Input(shape = (1,18), dtype = 'float32', name = 'Diag_Data_Input')
    hiddenLayer1 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_1')(diagInput)
  denseReshapeLayer = keras.layers.Lambda(lambda x: keras.backend.squeeze(x, 1),name = "Squeese")(hiddenLayer1)
  hiddenLayer2 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_2')(denseReshapeLayer)
  outputVarList = []
  outputLossFunctionDict = {}
  outputMetricDict = {}
  outputWeightList = []

  hiddenfeatureCount = 1024
  if bucketOutput:
    hiddenLayer3_LoS = keras.layers.Dense(hiddenfeatureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_3_for_multiclass')(hiddenLayer2)
    hiddenLayer4_LoS = keras.layers.Dense(hiddenfeatureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_4_for_multiclass')(hiddenLayer3_LoS)
    hiddenLayer5_LoS = keras.layers.Dense(hiddenfeatureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_5_for_multiclass')(hiddenLayer4_LoS)
    bucketDenseLayer = keras.layers.Dense(13,activation="softmax", name='Bucket_Output')(hiddenLayer5_LoS)
    outputVarList.append(bucketDenseLayer)
    outputLossFunctionDict['Bucket_Output'] = 'categorical_crossentropy'
    outputMetricDict['Bucket_Output'] = ['accuracy',keras.metrics.AUC(name='auc'),
                                         tfa.metrics.CohenKappa(num_classes=13,name = "kappa"),
                                         tf.keras.metrics.TopKCategoricalAccuracy(3, name = "top 3"),
                                         tf.keras.metrics.TopKCategoricalAccuracy(5, name = "top 5")]
    outputWeightList.append(1.)
  if numericOutput:
    hiddenLayer3_Num = keras.layers.Dense(hiddenfeatureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_3_for_numeric')(hiddenLayer2)
    hiddenLayer4_Num = keras.layers.Dense(hiddenfeatureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_4_for_numeric')(hiddenLayer3_Num)
    hiddenLayer5_Num = keras.layers.Dense(hiddenfeatureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_5_for_numeric')(hiddenLayer4_Num)
    numericDenseLayer = keras.layers.Dense(1,name='Numerical_Output',activation = 'relu')(hiddenLayer5_Num)
    outputVarList.append(numericDenseLayer)
    outputLossFunctionDict['Numerical_Output'] = 'mse'
    outputMetricDict['Numerical_Output'] = ['mean_absolute_error','mean_squared_error']
    outputWeightList.append(1.)
  if mortalityOutput:
    hiddenLayer3_Mor = keras.layers.Dense(hiddenfeatureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_3_for_Mortality')(hiddenLayer2)
    hiddenLayer4_Mor = keras.layers.Dense(hiddenfeatureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_4_for_Mortality')(hiddenLayer3_Mor)
    hiddenLayer5_Mor = keras.layers.Dense(hiddenfeatureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_5_for_Mortality')(hiddenLayer4_Mor)
    mortalityDenseLayer = keras.layers.Dense(1,activation='sigmoid',name='Mortality_Output')(hiddenLayer5_Mor)
    outputVarList.append(mortalityDenseLayer)
    outputLossFunctionDict['Mortality_Output'] = 'binary_crossentropy'
    outputMetricDict['Mortality_Output'] = ['accuracy',keras.metrics.AUC(name='auc')]
    outputWeightList.append(1.)
  inputVarList = []
  if includeVitals:
    inputVarList.extend([matrixFirstDayInput,matrixSecondDayInput])
  if includeAdmin:
    inputVarList.append(demoInput)
  if includePrescriptionPCAs:
    inputVarList.extend([prescriptionFirstDayInput,prescriptionSecondDayInput])
  if includeEmbeddings:
    inputVarList.append(embeddingInput)
  if includeDiag:
    inputVarList.append(diagInput) 
  rnnModel = keras.Model(inputs = inputVarList,
                         outputs = outputVarList)
  
  rnnModel.compile(optimizer='Adadelta', 
                   loss=outputLossFunctionDict, 
                   metrics=outputMetricDict, 
                   loss_weights=outputWeightList,)
  #tf.keras.utils.plot_model(rnnModel, show_shapes=True)
  return(rnnModel)



In [ ]:
def RNNRuns(includeAdmin = False, 
                 includeVitals = False, 
                  includeDiag = False,
                 includePrescriptionPCAs = False, 
                 includeEmbeddings = False,
                 bucketOutput = False, 
                 mortalityOutput = False, 
                 numericOutput = False, 
                 rnnMode = 'GRU'):

  model = RNNBuildModel(includeAdmin, 
                                 includeVitals, 
                                  includeDiag,
                                 includePrescriptionPCAs,
                                 includeEmbeddings, 
                                 bucketOutput, 
                                 mortalityOutput, 
                                 numericOutput, 
                                 rnnMode)
  #print(model.summary())
  inputVarList = []
  outputVarList = []
  TinputVarList = []
  ToutputVarList = []
  if includeVitals:
    inputVarList.extend([inputMatrixFirstDayArray,inputMatrixSecondDayArray])
    TinputVarList.extend([TinputMatrixFirstDayArray,TinputMatrixSecondDayArray])
  if includeAdmin:
    inputVarList.append(inputDemoArray)
    TinputVarList.append(TinputDemoArray)
  if includePrescriptionPCAs:
    inputVarList.extend([inputPrescriptionFirstDayArray,inputPrescriptionSecondDayArray])
    TinputVarList.extend([TinputPrescriptionFirstDayArray,TinputPrescriptionSecondDayArray])
  if includeEmbeddings:
    inputVarList.extend([inputNotesEmbeddingArray])
    TinputVarList.extend([TinputNotesEmbeddingArray])
  if includeDiag:
    inputVarList.extend([inputDiagArray])
    TinputVarList.extend([TinputDiagArray])
  if bucketOutput:
    outputVarList.append(outputLoSArray)
    ToutputVarList.append(ToutputLoSArray)
  if numericOutput:
    outputVarList.append(outputNumArray)
    ToutputVarList.append(ToutputNumArray)
  if mortalityOutput:
    outputVarList.append(outputBoolMortalArray)
    ToutputVarList.append(ToutputBoolMortalArray)
  if debugMode:
    seed(debugSeed)
    tfRandom.set_seed(debugSeed)

  if len(outputVarList) == 1:
    outputVarList = outputVarList[0]
  if len(inputVarList) == 1:
    inputVarList = inputVarList[0]

  earlyStopCallback = tf.keras.callbacks.EarlyStopping(monitor="val_loss",min_delta = 0, patience = 5, restore_best_weights = True)
  rnnHistory = model.fit(x=inputVarList,
                         y = outputVarList,
                         batch_size= 32, 
                         epochs= 128,
                         validation_split = 0.25,
                         verbose = 0, 
                         callbacks = earlyStopCallback,
                         shuffle = True)
  evalResult = model.evaluate(TinputVarList,ToutputVarList,verbose = 0, return_dict = True)
  
  return rnnHistory, evalResult

##Binary

In [ ]:
def RNNBuildBinaryModel(includeAdmin = False, 
                        includeDiag = False,
                        includeVitals = False, 
                        includePrescriptionPCAs = False,
                        includeEmbeddings = False,
                        rnnMode = 'GRU'):

  if not (includeAdmin or includeDiag or includeVitals or includePrescriptionPCAs or includeEmbeddings):
    return("Error: no input")

  tf.keras.backend.clear_session()
  if debugMode:
    seed(debugSeed)
    tfRandom.set_seed(debugSeed) 
  featureCount = 512

  if includeVitals:
    matrixFirstDayInput = keras.Input(shape= (12,9),dtype = 'float32',name = 'Vital_Signs_First_day_Input')
    matrixSecondDayInput = keras.Input(shape= (12,9),dtype = 'float32',name = 'Vital_Signs_Second_day_Input')
    #first day
    if rnnMode == 'GRU':
      firstDayRNNLayer = keras.layers.GRU(featureCount,input_shape = (12,9), name = "first_day_RNN_layer")(matrixFirstDayInput)
    else:
      firstDayRNNLayer = keras.layers.LSTM(featureCount,input_shape = (12,9), name = "first_day_RNN_layer")(matrixFirstDayInput)
    firstDaySDenseLayer = keras.layers.Dense(featureCount, name = "first_day_Dense_layer")(firstDayRNNLayer)
    firstDayReshapeLayer = keras.layers.Reshape((1,featureCount), name = "first_day_Reshape_layer")(firstDaySDenseLayer)
    #second day
    if rnnMode == 'GRU':
      secondDayRNNLayer = keras.layers.GRU(featureCount,input_shape = (12,9), name = "second_day_RNN_layer")(matrixSecondDayInput)
    else:
      secondDayRNNLayer = keras.layers.LSTM(featureCount,input_shape = (12,9), name = "second_day_RNN_layer")(matrixSecondDayInput)
    secondDaySDenseLayer = keras.layers.Dense(featureCount, name = "second_day_Dense_layer")(secondDayRNNLayer)
    secondDayReshapeLayer = keras.layers.Reshape((1,featureCount), name = "second_day_Reshape_layer")(secondDaySDenseLayer)
    #combine days
  if includePrescriptionPCAs:
    prescriptionFirstDayInput = keras.Input(shape = (1,7),dtype = "float32",name = 'Prescription_First_Day_Input')
    prescriptionSecondDayInput = keras.Input(shape = (1,7),dtype = "float32",name = 'Prescription_Second_Day_Input')
    if includeVitals:
      firstDayMergeLayer = keras.layers.concatenate([firstDayReshapeLayer,prescriptionFirstDayInput],name = 'first_Day_Prescription_Merge_Layer') #1 row
      secondDayMergeLayer = keras.layers.concatenate([secondDayReshapeLayer,prescriptionSecondDayInput],name = 'second_Day_Prescription_Merge_Layer') #1 row
      firstSecondDaymergeLayer = keras.layers.concatenate([firstDayMergeLayer,secondDayMergeLayer],name = 'first_Second_day_Merge_Layer', axis = 1) #2 rows
    else:
      firstSecondDaymergeLayer = keras.layers.concatenate([prescriptionFirstDayInput,prescriptionSecondDayInput],name = 'first_Second_day_Merge_Layer', axis = 1) #2 rows
  else:
    if includeVitals:
      firstSecondDaymergeLayer = keras.layers.concatenate([firstDayReshapeLayer,secondDayReshapeLayer],name = 'first_Second_day_Merge_Layer', axis = 1) #2 rows
  if includeEmbeddings:
    embeddingInput = keras.Input(shape = (None,781),dtype = 'float32',name = 'Embedding_Input')
    maskingLayer = keras.layers.Masking(mask_value = 0)(embeddingInput)
    if rnnMode == 'GRU':
      embeddingRNNLayer = keras.layers.GRU(featureCount, input_shape = (None,781),name = "Embedding_RNN_Layer")(maskingLayer)
    else:
      embeddingRNNLayer = keras.layers.LSTM(featureCount, input_shape = (None,781),name = "Embedding_RNN_Layer")(maskingLayer)
    embeddingReshapeLayer = keras.layers.Reshape((1,featureCount), name = "embedding_Reshape_layer")(embeddingRNNLayer)
  if (includeVitals or includePrescriptionPCAs):
    if rnnMode == 'GRU':
      combinedRNNLayer = keras.layers.GRU(featureCount, name = "combined_RNN_layer")(firstSecondDaymergeLayer)
    else:
      combinedRNNLayer = keras.layers.LSTM(featureCount, name = "combined_RNN_layer")(firstSecondDaymergeLayer)
    combinedDenseLayer = keras.layers.Dense(featureCount,name = "combined_Dense_layer")(combinedRNNLayer)
    combinedReshapeLayer = keras.layers.Reshape((1,featureCount), name = "combined_Reshape_layer")(combinedDenseLayer)
    #hiddenLayer1 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_1')(combinedReshapeLayer)
    if includeAdmin:
      demoInput = keras.Input(shape = (1,73),dtype = "float32",name = 'Admin_Data_Input')
      adminDataRNNMergeLayer = keras.layers.concatenate([combinedReshapeLayer,demoInput],name = 'admin_Data_RNN_Merge_Layer')
      if includeEmbeddings:
        if includeDiag:
          diagInput = keras.Input(shape = (1,18), dtype = 'float32', name = 'Diag_Data_Input')
          diagMergeLayer = keras.layers.concatenate([adminDataRNNMergeLayer,diagInput], name = 'diag_Merge_Layer')
          embeddingMergeLayer = keras.layers.concatenate([diagMergeLayer,embeddingReshapeLayer], name = 'embedding_Merge_Layer')
        else:
          embeddingMergeLayer = keras.layers.concatenate([adminDataRNNMergeLayer,embeddingReshapeLayer], name = 'embedding_Merge_Layer')
        hiddenLayer1 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_1')(embeddingMergeLayer)
      else:
        if includeDiag:
          diagInput = keras.Input(shape = (1,18), dtype = 'float32', name = 'Diag_Data_Input')
          diagMergeLayer = keras.layers.concatenate([adminDataRNNMergeLayer,diagInput], name = 'diag_Merge_Layer')
          hiddenLayer1 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_1')(diagMergeLayer)
        else:
          hiddenLayer1 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_1')(adminDataRNNMergeLayer)
    else:
      if includeEmbeddings:
        if includeDiag:
          diagInput = keras.Input(shape = (1,18), dtype = 'float32', name = 'Diag_Data_Input')
          diagMergeLayer = keras.layers.concatenate([combinedReshapeLayer,diagInput], name = 'diag_Merge_Layer')
          embeddingMergeLayer = keras.layers.concatenate([diagMergeLayer,embeddingReshapeLayer], name = 'embedding_Merge_Layer')
        else:
          embeddingMergeLayer = keras.layers.concatenate([combinedReshapeLayer,embeddingReshapeLayer], name = 'embedding_Merge_Layer')
        hiddenLayer1 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_1')(embeddingMergeLayer)
      else:
        if includeDiag:
          diagInput = keras.Input(shape = (1,18), dtype = 'float32', name = 'Diag_Data_Input')
          diagMergeLayer = keras.layers.concatenate([combinedReshapeLayer,diagInput], name = 'diag_Merge_Layer')
          hiddenLayer1 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_1')(diagMergeLayer)
        else:
          hiddenLayer1 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_1')(combinedReshapeLayer)
  else:
      demoInput = keras.Input(shape = (1,73),dtype = "float32",name = 'Admin_Data_Input')
      if includeEmbeddings:
        if includeDiag:
          diagInput = keras.Input(shape = (1,18), dtype = 'float32', name = 'Diag_Data_Input')
          diagMergeLayer = keras.layers.concatenate([demoInput,diagInput], name = 'diag_Merge_Layer')
          embeddingMergeLayer = keras.layers.concatenate([diagMergeLayer ,embeddingReshapeLayer], name = 'embedding_Merge_Layer')
        else:
          embeddingMergeLayer = keras.layers.concatenate([demoInput,embeddingReshapeLayer], name = 'embedding_Merge_Layer')
        hiddenLayer1 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_1')(embeddingMergeLayer)
      else:
        if includeDiag:
          diagInput = keras.Input(shape = (1,18), dtype = 'float32', name = 'Diag_Data_Input')
          diagMergeLayer = keras.layers.concatenate([demoInput,diagInput], name = 'diag_Merge_Layer')
          hiddenLayer1 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_1')(diagMergeLayer)
        else:
          hiddenLayer1 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_1')(demoInput)
  if includeEmbeddings and not includeAdmin and not includePrescriptionPCAs and not includeVitals:
    if includeDiag:
      diagInput = keras.Input(shape = (1,18), dtype = 'float32', name = 'Diag_Data_Input')
      diagMergeLayer = keras.layers.concatenate([embeddingReshapeLayer,diagInput], name = 'diag_Merge_Layer')
      hiddenLayer1 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_1')(diagMergeLayer)
    else:
      hiddenLayer1 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_1')(embeddingReshapeLayer)
  if includeDiag and not includeEmbeddings and not includeAdmin and not includePrescriptionPCAs and not includeVitals:
    diagInput = keras.Input(shape = (1,18), dtype = 'float32', name = 'Diag_Data_Input')
    hiddenLayer1 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_1')(diagInput)
  denseReshapeLayer = keras.layers.Lambda(lambda x: keras.backend.squeeze(x, 1),name = "Squeese")(hiddenLayer1)
  hiddenLayer2 = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_2')(denseReshapeLayer)
  hiddenLayer3_Bin = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_3_for_multiclass')(hiddenLayer2)
  hiddenLayer4_Bin = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_4_for_multiclass')(hiddenLayer3_Bin)
  hiddenLayer5_Bin = keras.layers.Dense(featureCount,use_bias=True,activation=activationOption,name = 'Dense_Layer_5_for_multiclass')(hiddenLayer4_Bin)
  binaryDenseLayer = keras.layers.Dense(1,activation="sigmoid", name='Bucket_Output')(hiddenLayer5_Bin)
  inputVarList = []
  if includeVitals:
    inputVarList.extend([matrixFirstDayInput,matrixSecondDayInput])
  if includeAdmin:
    inputVarList.append(demoInput)
  if includePrescriptionPCAs:
    inputVarList.extend([prescriptionFirstDayInput,prescriptionSecondDayInput])
  if includeEmbeddings:
    inputVarList.append(embeddingInput)
  if includeDiag:
    inputVarList.append(diagInput)
  
  cnnModel = keras.Model(inputs = inputVarList,
                         outputs = binaryDenseLayer)
  
  cnnModel.compile(optimizer='Adadelta', 
                   loss='binary_crossentropy', 
                   metrics=["accuracy", keras.metrics.AUC(name='auc'),'mae','mse'])
  return(cnnModel)



In [ ]:
def RNNBinaryRuns(outputVar, ToutputVar,
    includeAdmin = False, 
                 includeVitals = False, 
                  includeDiag = False,
                 includePrescriptionPCAs = False, 
                 includeEmbeddings = False,
                 rnnMode = 'GRU'
                 ):

  model = RNNBuildBinaryModel(includeAdmin,
                              includeDiag,
                                 includeVitals, 
                                 includePrescriptionPCAs, 
                              includeEmbeddings,rnnMode = 'GRU')
  #print(model.summary())
  inputVarList = []
  TinputVarList = []
  if includeVitals:
    inputVarList.extend([inputMatrixFirstDayArray,inputMatrixSecondDayArray])
    TinputVarList.extend([TinputMatrixFirstDayArray,TinputMatrixSecondDayArray])
  if includeAdmin:
    inputVarList.append(inputDemoArray)
    TinputVarList.append(TinputDemoArray)
  if includePrescriptionPCAs:
    inputVarList.extend([inputPrescriptionFirstDayArray,inputPrescriptionSecondDayArray])
    TinputVarList.extend([TinputPrescriptionFirstDayArray,TinputPrescriptionSecondDayArray])
  if includeEmbeddings:
    inputVarList.extend([inputNotesEmbeddingArray])
    TinputVarList.extend([TinputNotesEmbeddingArray])
  if includeDiag:
    inputVarList.extend([inputDiagArray])
    TinputVarList.extend([TinputDiagArray])
  if debugMode:
    seed(debugSeed)
    tfRandom.set_seed(debugSeed)
  if len(inputVarList) == 1:
    inputVarList = inputVarList[0]
    TinputVarList = TinputVarList[0]

  earlyStopCallback = tf.keras.callbacks.EarlyStopping(monitor="val_loss",min_delta = 0, patience = 5, restore_best_weights = True)
  rnnHistory = model.fit(x=inputVarList,
                         y = outputVar,
                         batch_size= 32, 
                         epochs= 128,
                         validation_split = 0.25,
                         verbose = 0, 
                         callbacks = earlyStopCallback,
                         shuffle = True)
  evalResult = model.evaluate(TinputVarList,ToutputVar,verbose = 0, return_dict = True)
  if doingTests:
  #SAPS
    tempList1 = []
    tempList2 = []
    for i in range(0,TinputSAPSArray.shape[0]):
      if TinputSAPSArray[i] > 54.0:
        tempList2.append(i)
      else:
        tempList1.append(i)    
    medianList = [tempList1,tempList2]
  #OASIS
    tempList1 = []
    tempList2 = []
    for i in range(0,TinputOASISArray.shape[0]):
      if TinputOASISArray[i] > 38.0:
        tempList2.append(i)
      else:
        tempList1.append(i)
    medianList.extend([tempList1,tempList2])
  #SOFA
    tempList1 = []
    tempList2 = []
    for i in range(0,TinputSOFAArray.shape[0]):
      if TinputSOFAArray[i] > 7.0:
        tempList2.append(i)
      else:
        tempList1.append(i)
    medianList.extend([tempList1,tempList2])
  #AGE
    tempList1 = []
    tempList2 = []
    for i in range(0,TinputDemoArray.shape[0]):
      if TinputDemoArray[i,0,0] >= 18.0:
        tempList2.append(i)
      else:
        tempList1.append(i)
    medianList.extend([tempList1,tempList2])

  #comorbidities
    tempList1 = []
    tempList2 = []
    tempList3 = []
    tempList4 = []
    tempList5 = []
    tempList6 = []
    tempList7 = []
    tempList8 = []
    tempList9 = []
    tempList10 = []
    tempList11 = []

    for i in range(0,TinputComorbidityArray.shape[0]):
      #no comorbidity
      if sum(TinputComorbidityArray[i,0,]) == 0:
        tempList1.append(i)
      #congestive_heart_failure
      if TinputComorbidityArray[i,0,0] == 1:
        tempList2.append(i)
      #cardiac_arrhythmias
      if TinputComorbidityArray[i,0,1] == 1:
        tempList3.append(i)
      #chronic_pulmonary 
      if TinputComorbidityArray[i,0,2] == 1:
        tempList4.append(i)
      #diabetes_uncomplicated
      if TinputComorbidityArray[i,0,3] == 1:
        tempList5.append(i)
      #renal_failure
      if TinputComorbidityArray[i,0,4] == 1:
        tempList6.append(i)
      #coagulopathy
      if TinputComorbidityArray[i,0,5] == 1:
        tempList7.append(i)
      #fluid_electrolyte
      if TinputComorbidityArray[i,0,6] == 1:
        tempList8.append(i)
      #deficiency_anemias
      if TinputComorbidityArray[i,0,7] == 1:
        tempList9.append(i)
      if TinputComorbidityArray[i,0,8] == 1:
        tempList10.append(i)
      if TinputComorbidityArray[i,0,9] == 1:
        tempList11.append(i)



    medianList.extend([tempList1,tempList2,tempList3,tempList4,tempList5,tempList6,tempList7,tempList8,tempList9,tempList10,tempList11])

    IOList = seperatIO(TinputVarList,ToutputVar,medianList)
    testList = []
    for i in IOList:
      output = i[-1]
      input = i[:-1]
      #print("Test data sub group")
      tempMap = model.evaluate(input,output,verbose = 0, return_dict = True)
      tempMap["TrueValue"] = np.average(output)
      testList.append(tempMap)
    evalResult["tests"] = testList
    del IOList
  del model
  gc.collect()
  return rnnHistory, evalResult

#Random Forest Bootstrap Baseline

In [ ]:
def randomForestRuns(includeAdmin = False, includeVitals = False, includePrescriptionPCAs = False):
  if not (includeAdmin or includeVitals or includePrescriptionPCAs):
    return("error, no input")

  randomForestInputList = []
  randomForestBucketOutputList = []
  randomForestMortalityOutputList = []
  randomForestNumericOutputList = []
  for i in range(0,len(inputDemoList)):
    temp = []
    if includeAdmin:
      temp.extend(inputDemoArray[i].flatten())
    if includeVitals:
      temp.extend(inputMatrixArray[i].flatten())
    if includePrescriptionPCAs:
      temp.extend(inputPrescriptionFirstDayArray[i].flatten())
      temp.extend(inputPrescriptionSecondDayArray[i].flatten())
    randomForestInputList.append(temp)
    los = math.ceil(outputNumList[i])
    if los>13:
      los = 13
    randomForestBucketOutputList.append(str(los))
    randomForestMortalityOutputList.append(outputBoolMortalArray[i].flatten())
    randomForestNumericOutputList.append(outputNumList[i])

  TrandomForestInputList = []
  TrandomForestBucketOutputList = []
  TrandomForestMortalityOutputList = []
  TrandomForestNumericOutputList = []
  pbar = ProgressBar()
  for i in range(0,len(TinputDemoList)):
    temp = []
    if includeAdmin:
      temp.extend(TinputDemoArray[i].flatten())
    if includeVitals:
      temp.extend(TinputMatrixArray[i].flatten())
    if includePrescriptionPCAs:
      temp.extend(TinputPrescriptionFirstDayArray[i].flatten())
      temp.extend(TinputPrescriptionSecondDayArray[i].flatten())
    TrandomForestInputList.append(temp)
    los = math.ceil(outputNumList[i])
    if los>13:
      los = 13
    TrandomForestBucketOutputList.append(str(los))
    TrandomForestMortalityOutputList.append(outputBoolMortalArray[i].flatten())
    TrandomForestNumericOutputList.append(outputNumList[i])
  returnDict = {}
  accMor, aucMor = randomForestMortality(randomForestInputList, randomForestMortalityOutputList, TrandomForestInputList, TrandomForestMortalityOutputList)
  returnDict['accMor'] = accMor
  returnDict['aucMor'] = aucMor
  accLoS, aucLoS, kapLoS = randomForestBuckets(randomForestInputList, randomForestBucketOutputList, TrandomForestInputList, TrandomForestBucketOutputList)
  returnDict['accLoS'] = accLoS
  returnDict['aucLoS'] = aucLoS
  returnDict['kapLoS'] = kapLoS
  MAENum, MSENum = randomForestNumeric(randomForestInputList, randomForestNumericOutputList, TrandomForestInputList, TrandomForestNumericOutputList)
  returnDict['MSENum'] = MSENum
  returnDict['MAENum'] = MAENum
 
  return(returnDict)

def randomForestMortality(randomForestInputList, randomForestMortalityOutputList, TrandomForestInputList, TrandomForestMortalityOutputList):
  binaryModel = RandomForestClassifier()
  binaryModel.fit(randomForestInputList,randomForestMortalityOutputList)
  predictMor = binaryModel.predict(TrandomForestInputList)
  accMor = sklearn.metrics.accuracy_score(predictMor, TrandomForestMortalityOutputList)
  aucMor = sklearn.metrics.roc_auc_score(TrandomForestMortalityOutputList, binaryModel.predict_proba(TrandomForestInputList)[:,1])
  return(accMor,aucMor)

def randomForestBuckets(randomForestInputList, randomForestBucketOutputList, TrandomForestInputList, TrandomForestBucketOutputList):
  bucketModel = OneVsRestClassifier(RandomForestClassifier(n_estimators = 100, criterion = 'entropy',n_jobs=-1,max_depth = 5))
  bucketModel.fit(randomForestInputList, randomForestBucketOutputList)
  predictLoS = bucketModel.predict(TrandomForestInputList)
  accLoS = sklearn.metrics.accuracy_score(predictLoS, TrandomForestBucketOutputList)
  aucLoS = sklearn.metrics.roc_auc_score(TrandomForestBucketOutputList, bucketModel.predict_proba(TrandomForestInputList), multi_class = 'ovr')
  kapLoS = sklearn.metrics.cohen_kappa_score(predictLoS, TrandomForestBucketOutputList)
  return(accLoS, aucLoS, kapLoS)

def randomForestNumeric(randomForestInputList, randomForestNumericOutputList, TrandomForestInputList, TrandomForestNumericOutputList):
  numericModel = RandomForestRegressor(n_estimators = 100,n_jobs=-1,max_depth = 5)
  numericModel.fit(randomForestInputList, randomForestNumericOutputList)
  predictNum = numericModel.predict(TrandomForestInputList)
  MSENum = sklearn.metrics.mean_squared_error(TrandomForestNumericOutputList, predictNum)
  MAENum = sklearn.metrics.mean_absolute_error(TrandomForestNumericOutputList, predictNum)
  return(MAENum, MSENum)

#BinaryBaseLine


In [ ]:
def randomForestMortality(randomForestInputList, randomForestMortalityOutputList, TrandomForestInputList, TrandomForestMortalityOutputList):
  binaryModel = RandomForestClassifier()
  binaryModel.fit(randomForestInputList,randomForestMortalityOutputList)
  predictMor = binaryModel.predict(TrandomForestInputList)
  accMor = sklearn.metrics.accuracy_score(predictMor, TrandomForestMortalityOutputList)
  aucMor = sklearn.metrics.roc_auc_score(TrandomForestMortalityOutputList, binaryModel.predict_proba(TrandomForestInputList)[:,1])
  return(accMor,aucMor)


def binaryRunBase(binaryDate = 3, runCount = 10):
  pbar = ProgressBar()
  rfBinaryACCList = []
  rfBinaryAUCList = []
  rfBinaryMAEList = []
  rfBinaryMSEList = []
  for i in pbar(range(0,runCount)):
    x,y = binarySingleRunRandomForest(binaryDate)
    #print(x)
    #print(y)
    rfBinaryACCList.append(x)
    rfBinaryAUCList.append(y)

  print("Baseline random forest model:")
  print("-----Binary Accuracy for exiting on %d day: " % binaryDate)
  print("Mean: %.3f //// Stdev: %.3f" % (statistics.mean(rfBinaryACCList), statistics.stdev(rfBinaryACCList)))

  print("-----Binary AUC for exiting on %d day: " % binaryDate)
  print("Mean: %.3f //// Stdev: %.3f" % (statistics.mean(rfBinaryAUCList), statistics.stdev(rfBinaryAUCList)))

  #print("Baseline random forest model:")
  #print("-----MAE: ")
  #print("Mean: %.3f //// Stdev: %.3f" % (statistics.mean(rfBinaryMAEList), statistics.stdev(rfBinaryMAEList)))

  #print("-----MSE: ")
  #print("Mean: %.3f //// Stdev: %.3f" % (statistics.mean(rfBinaryMSEList), statistics.stdev(rfBinaryMSEList)))

In [ ]:
def binarySingleRunRandomForest(binaryDate = 3):
  reshuffle()

  outputVarList = []
  ToutputVarList = []
  if(binaryDate > 15 or binaryDate < 3):
    print("error")
    return()

  for i in outputNumArray:
    if i > (binaryDate - 2):
      outputVarList.append(False)
    else:
      outputVarList.append(True)
  for i in ToutputNumArray:
    if i > (binaryDate - 2):
      ToutputVarList.append(False)
    else:
      ToutputVarList.append(True)
  
  outputVarList = np.array(outputVarList)
  ToutputVarList = np.array(ToutputVarList)

  inputVarList = []
  TinputVarList = []
  for i in range(0,len(outputVarList)):
      temp = []
      temp.extend(inputDemoArray[i].flatten())
      temp.extend(inputDiagArray[i].flatten())
      temp.extend(inputMatrixArray[i].flatten())
      temp.extend(inputPrescriptionFirstDayArray[i].flatten())
      temp.extend(inputPrescriptionSecondDayArray[i].flatten())
      inputVarList.append(temp)
  for i in range(0,len(ToutputVarList)):
      temp = []
      temp.extend(TinputDemoArray[i].flatten())
      temp.extend(TinputDiagArray[i].flatten())
      temp.extend(TinputMatrixArray[i].flatten())
      temp.extend(inputPrescriptionFirstDayArray[i].flatten())
      temp.extend(inputPrescriptionSecondDayArray[i].flatten())
      TinputVarList.append(temp)
  #print(randomForestNumeric(inputVarList, outputNumArray,TinputVarList, ToutputNumArray))
  return(randomForestMortality(inputVarList, outputVarList, TinputVarList, ToutputVarList))

# Time & Accuracy Test

In [ ]:
binaryRun(True,True,True,True,True,3,5)

In [ ]:
binaryRun(True,True,True,True,True,7,5)

In [ ]:
binaryRun(True,True,True,True,True,14,5)

In [ ]:
RunModelsNumericOutput(True,True,True,True,True,5)